<h1><center> Projeto PCD </center></h1>
<center> Autores: Bruna Guedes; Jônatas de Oliveira; Luiza Davoli; Matheus Macedo </center>

## Sumário

1. Introdução  
2. Bibliotecas

3. Funções

    3.1. numero_genomas

    3.2. seleção_virus

    3.3. buscar_genoma

    3.4. baixar_genoma

    3.5. rodar_muscle

    3.6. plotar_matriz_triangular

3. Referências  


## Introdução

## Bibliotecas

Abaixo estão listadas todas as bibliotecas utilizadas em nosso código:

In [ ]:
from Bio import Entrez, SeqIO
from pathlib import Path
import subprocess
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from tkinter import *
from tkinter import ttk

## Funções

Abaixo definimos todas as funções necessárias para o funcionamento do código:

`numero_genomas` : Essa função cria uma caixa interativa, onde é possível digitar a quantidade de genomas que se deseja comparar

In [6]:
def numero_genomas():

    root = Tk()

    # Cria a caixa de texto em que o usuário deve informar a quantidade de genomas que deseja comparar
    content = ttk.Frame(root, padding=(3,3,12,12))
    numerolbl = ttk.Label(content, text="Quantidade de genomas a se comparar")
    numero = ttk.Entry(content)

    # Cria botões interativos para que o usuário possa confirmar ou cancelar sua escolha
    ok = ttk.Button(content, text="OK")
    cancelar = ttk.Button(content, text="Cancelar")

    # Configurações gráficas da 
    content.grid(column=0, row=0, sticky=(N, S, E, W))
    numerolbl.grid(column=2, row=0, columnspan=3, sticky=(N, W), padx=5)
    numero.grid(column=1, row=1, columnspan=4, sticky=(N,E,W), pady=5, padx=5)
    ok.grid(column=3, row=3)
    cancelar.grid(column=4, row=3)

    root.columnconfigure(0, weight=2)
    root.rowconfigure(0, weight=2)
    content.columnconfigure(0, weight=4)
    content.columnconfigure(1, weight=4)
    content.columnconfigure(2, weight=4)
    content.columnconfigure(3, weight=2)
    content.columnconfigure(4, weight=2)
    content.rowconfigure(1, weight=2)

    root.mainloop()

    return numero

`seleção_virus` : Essa função cria uma caixa interativa, onde é possível digitar o nome do vírus que se deseja procurar o genoma. A variável `nome_virus` salva os vírus selecionados, para que se possa realizar a busca no NCBI.

In [7]:
def seleção_virus():

    root = Tk()

    content = ttk.Frame(root, padding=(3,3,12,12))
    namelbl = ttk.Label(content, text="Nome do Vírus")
    nome_virus = ttk.Entry(content)

    ok = ttk.Button(content, text="OK")
    cancelar = ttk.Button(content, text="Cancelar")

    content.grid(column=0, row=0, sticky=(N, S, E, W))
    namelbl.grid(column=2, row=0, columnspan=3, sticky=(N, W), padx=5)
    nome_virus.grid(column=1, row=1, columnspan=4, sticky=(N,E,W), pady=5, padx=5)
    ok.grid(column=3, row=3)
    cancelar.grid(column=4, row=3)

    root.columnconfigure(0, weight=2)
    root.rowconfigure(0, weight=2)
    content.columnconfigure(0, weight=4)
    content.columnconfigure(1, weight=4)
    content.columnconfigure(2, weight=4)
    content.columnconfigure(3, weight=2)
    content.columnconfigure(4, weight=2)
    content.rowconfigure(1, weight=2)

    root.mainloop()

    return nome_virus

`buscar_genoma` : Essa função busca o vírus selecionado na base de dados do NCBI (National Center for Biotechnology Information). Para isso, a função recebe o nome do vírus, busca o ID correspondente e retorna essa informação ao usuário. Esse é um passo muito importante, pois um mesmo vírus pode ter diferentes cepas e sequenciamentos, os quais terão diferentes IDs no NCBI. Caso o vírus não esteja na base de dados do site, a função informa o usuário sobre esse problema e requisita uma nova tentativa

In [ ]:
def buscar_genoma(nome_virus):
    Entrez.email = "computandocombiopy@gmail.com"
    termo_de_busca = f"{nome_virus} complete genome"
    busca = Entrez.esearch (db = "nucleotide", term = termo_de_busca, retmax = 1)
    resultado = Entrez.read(busca)
    busca.close()
    
    id_do_virus = (resultado["IdList"])

    if not id_do_virus:
        print(f"Não há um vírus {nome_virus} no NCBI. Tente outra vez!")

    return id_do_virus[0]

`baixar_genoma` : Essa função será utilizada para baixar os genomas dos vírus escolhidos pelo usuário. 

In [ ]:
def baixar_genoma(iden, output_folder=Path("data")/"genomes"):
    
    # Faz os genomas irem para uma pasta relativa ao repositório
    output_folder.mkdir(parents=True, exist_ok=True)
    filename= output_folder/f"{iden}.fasta"
    
    # Acessando o Entrez
    Entrez.email="computandocombiopy@gmail.com"

    # Checa se o arquivo fasta existe, se não, ele o cria
    if not filename.exists():
        
        #Baixa o arquivo .fasta (funciona para GenBank e RefSeq)
        stream= Entrez.efetch(
            db="nucleotide",id=iden, rettype="fasta", retmode="text"
        )
        data = stream.read()
        
        #Checa se data tá vazio ou não
        if not data:
            print("Error: Resposta vazia do NCBI.")
            
        else:
            with open(filename, "w") as output:
                output.write(data)
        print(f"Salvo em {filename}")

`rodar_muscle` : Função que roda o muscle, utilizado para realizar o alinhamento entre genosmas de diferentes vírus, a fim de compará-los.

In [ ]:
def rodar_muscle(entrada_fasta, saida_fasta): #recebe 2 arquivos como parâmetros, um genomas.fasta e outro genomas_alinhados.fasta (onde o MUSCLE salva o resultado do alinhamento múltiplo)
    
    # Acessa o executável do MUSCLE
    muscle_exe = "muscle-win64.v5.3.exe"
    
    # Alinha o arquivo de entrada e coloca os dados de saída no arquivo criado pelo executável, no caso genomas_alinhados.fasta, que foi passado como parâmetro
    comando = [muscle_exe, "-align", entrada_fasta, "-output", saida_fasta]
    
    # Tratamento de erro
    try: 
        resultado = subprocess.run(comando, check=True, capture_output=True, text=True) 
        print("MUSCLE finalizado com sucesso.")
        print("STDOUT:", resultado.stdout)
        print("STDERR:", resultado.stderr)
    except subprocess.CalledProcessError as e:
        print("Erro ao rodar MUSCLE:")
        print("STDOUT:", e.stdout)
        print("STDERR:", e.stderr)
        
        # Imprime o erro novamente, além de interromper o programa, impedindo que o erro se prolongue
        raise 

`plotar_matriz_triangular` : Cria uma matriz triangular, representando as semelhanças e diferenças entre os vírus escolhidos. Com isso, gera um gráfico, possibilitando um entendimento mais visual.

In [ ]:
def plotar_matriz_triangular(nomes, matriz): #recebe uma lista de nomes e uma matriz de similaridade, com valores entre 0 e 1
    matriz_np = np.array(matriz) #garantindo que a matriz é um array NumPy, necessário para facilitar as operações matemáticas e criação de gráficos

    # Máscara para esconder a parte superior da matriz (acima da diagonal)
    mask = np.triu(np.ones_like(matriz_np, dtype=bool))

    plt.figure(figsize=(8, 6)) #criando figura com tamanho definido em polegadas (largura, altura)
    sns.heatmap(matriz_np, annot=True, fmt=".2f", mask=mask,
                xticklabels=nomes, yticklabels=nomes,
                cmap="YlGnBu", linewidths=0.5, cbar_kws={"label": "Similaridade"},
                vmin=0.001, vmax=1.0)

    plt.title("Matriz de Similaridade Genômica", fontsize=16)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()

## Referências:

Sequenciamento genômico: no rastro do vírus. Disponível em: <https://agencia.fiocruz.br/sequenciamento-genomico-no-rastro-do-virus>. Acesso em: 11 jun. 2025.

Accessing NCBI’s Entrez databases — Biopython 1.85 documentation. Disponível em: <https://biopython.org/docs/latest/Tutorial/chapter_entrez.html#chapter-entrez>. Acesso em: 11 jun. 2025.

NATIONAL CENTER FOR BIOTECHNOLOGY INFORMATION. Our Mission - NCBI. Disponível em: <https://www.ncbi.nlm.nih.gov/home/about/mission/>.

‌DOS, C. GenBank. Disponível em: <https://pt.wikipedia.org/wiki/GenBank>. Acesso em: 11 jun. 2025.

‌BIOPYTHON. Biopython · Biopython. Disponível em: <https://biopython.org/>.

Biopython Documentation — Biopython 1.85 documentation. Disponível em: <https://biopython.org/docs/latest/index.html#>. Acesso em: 11 jun. 2025.

‌Cock, P. J. A., Antao, T., Chang, J. T., Chapman, B. A., Cox, C. J., Dalke, A., Friedberg, I., Hamelryck, T., Kauff, F., Wilczynski, B., & de Hoon, M. J. L. (2009). Biopython: freely available Python tools for computational molecular biology and bioinformatics. Bioinformatics, 25(11), 1422–1423.

‌OPENAI. ChatGPT (versão GPT-4) [ferramenta de inteligência artificial]. Disponível em: https://chat.openai.com/. Acesso em: 11 jun. 2025.

WIKIPEDIA CONTRIBUTORS. Sequence alignment. Disponível em: <https://en.wikipedia.org/wiki/Sequence_alignment>.

WIKIPEDIA CONTRIBUTORS. MUSCLE (alignment software).

INTRODUÇÃO, 3. 1. Alinhamento de Seqüências. Disponível em: <https://professor.pucgoias.edu.br/SiteDocente/admin/arquivosUpload/18497/material/Cap.%203%20Alinhamento%20de%20sequ%C3%AAncias.pdf>. Acesso em: 11 jun. 2025.

The module for multiple sequence alignments, AlignIO. Disponível em: <https://biopython.org/wiki/AlignIO>. Acesso em: 11 jun. 2025.

Introduction to NumPy. Disponível em: <https://www.w3schools.com/python/numpy/numpy_intro.asp>. Acesso em: 11 jun. 2025.



Edgar, R. C. (2004). MUSCLE: a multiple sequence alignment method with reduced time and space complexity. BMC Bioinformatics, 5, 113. 


TkDocs home. Disponível em: <https://tkdocs.com/index.html>. Acesso em: 18 jun. 2025.